In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../fixation/18sat_fixfinal.csv")
df_filtered = df[(df['page_name']== 'reading-dickens-1') & (df['RECORDING_SESSION_LABEL'] == 'msd001')]
df_filtered = df_filtered.sort_values("CURRENT_FIX_INTEREST_AREA_ID")

In [23]:
label = pd.read_csv("../fixation/18sat_labels.csv")

In [27]:
df_filtered[['RECORDING_SESSION_LABEL', 'CURRENT_FIX_X', 'CURRENT_FIX_Y', 'CURRENT_FIX_PUPIL', 'CURRENT_FIX_DURATION']] \
    .merge(label[label['book'] == 'dickens'], left_on='RECORDING_SESSION_LABEL', right_on='subj', how = 'inner' )

,RECORDING_SESSION_LABEL,CURRENT_FIX_X,CURRENT_FIX_Y,CURRENT_FIX_PUPIL,CURRENT_FIX_DURATION,subj,book,acc,confidence,difficulty,...,interest,pressured,sleepiness,sleephours,acc_level,subj_acc,language,sex,native,subj_acc_level
0,msd001,819.9,718.7,1673.0,213,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
1,msd001,867.4,708.7,1519.0,165,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
2,msd001,167.3,124.8,1348.0,194,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
3,msd001,225.6,114.6,1358.0,117,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
4,msd001,270.9,111.3,1382.0,210,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,msd001,697.0,645.5,1785.0,183,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
108,msd001,798.9,640.4,1606.0,255,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
109,msd001,864.5,639.3,1567.0,143,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
110,msd001,59.8,125.4,1430.0,22,msd001,dickens,0.6,2,0,...,2,0,1,2,2,0.75,English,F,1,3
